In [ ]:
from pathlib import Path
import numpy as np
import proplot as pplt
import xarray as xr
from plotting import OIFSPreprocessor, NEMOPreprocessor

In [ ]:
oifs_preprocessor = OIFSPreprocessor(
    np.datetime64("2014-07-01"), np.timedelta64(-7, "h")
)
nemo_preprocessor = NEMOPreprocessor(np.timedelta64(-7, "h"))

In [ ]:
exp_id = "RAS0"
plot_folder = Path(f"plots/{exp_id}")
plot_folder.mkdir(exist_ok=True)
max_iters = 25
alpha = 0.25

In [ ]:
oifs_diagvars = [
    xr.open_mfdataset(
        f"PAPA/{exp_id}_{iter}/diagvar.nc", preprocess=oifs_preprocessor.preprocess
    )
    for iter in range(1, max_iters + 1)
]
oifs_progvars = [
    xr.open_mfdataset(
        f"PAPA/{exp_id}_{iter}/progvar.nc", preprocess=oifs_preprocessor.preprocess
    )
    for iter in range(1, max_iters + 1)
]
nemo_t_grids = [
    xr.open_mfdataset(
        f"PAPA/{exp_id}_{iter}/{exp_id}_*_T.nc", preprocess=nemo_preprocessor.preprocess
    )
    for iter in range(1, max_iters + 1)
]

In [ ]:
fig, ax = pplt.subplots()
fig.set_size_inches(10, 6)
for oifs_diagvar in oifs_diagvars[1:]:
    ax.plot(oifs_diagvar.sfc_swrad, ls="-", alpha=alpha, color="k")
ax.plot(oifs_diagvars[0].sfc_swrad, ls="-", alpha=alpha)
ax.format(
    title="Surface SW Radiation (OIFS)",
    xlabel="Time",
    ylabel="Radiative Flux $[W m^{-2}]$",
)
fig.savefig(f"plots/{exp_id}/ssw_oifs.pdf", bbox_inches="tight")
pplt.close(fig)

In [ ]:
fig, ax = pplt.subplots()
fig.set_size_inches(10, 6)
for oifs_progvar in oifs_progvars[1:]:
    ax.plot(oifs_progvar.t[:, 59] - 273.15, ls="-", alpha=alpha, color="k")
ax.plot(oifs_progvars[0].t[:, 59] - 273.15, ls="-", alpha=alpha)
ax.set_title("Temperature at 10m (OIFS)")
ax.format(title="Temperature at 10m (OIFS)", xlabel="Time", ylabel="Temperature [°C]")
fig.savefig(f"plots/{exp_id}/10t_oifs.pdf", bbox_inches="tight")
pplt.close(fig)

In [ ]:
oifs_progvar_cvg = oifs_progvars[-1]
errors = [
    np.linalg.norm((oifs_progvar.t - oifs_progvar_cvg.t).data, ord=np.inf)
    for oifs_progvar in oifs_progvars[:-1]
]
fig, ax = pplt.subplots()
fig.set_size_inches(10, 6)
x_data = np.arange(1, max_iters)
ax.semilogy(x_data, errors, color="k", ls="none", marker=".")
ax.format(
    title="Errors in 10m Temperature (OIFS)",
    xlabel="Schwarz Iteration",
    ylabel=r"$||e||_\infty$",
    xlocator=x_data[::2],
    xminorlocator=1,
)
fig.savefig(
    f"plots/{exp_id}/errors_linf_10t_oifs.pdf",
    bbox_inches="tight",
)
pplt.close(fig)

In [ ]:
fig, ax = pplt.subplots()
fig.set_size_inches(10, 6)
for nemo_t_grid in nemo_t_grids[1:]:
    ax.plot(nemo_t_grid.sosstsst[:, 1, 1], ls="-", alpha=alpha, color="k")
ax.plot(nemo_t_grids[0].sosstsst[:, 1, 1], ls="-", alpha=alpha)
ax.format(
    title="Sea Surface Temperature (NEMO)", ylabel="Temperature [°C]", xlabel="Time"
)
fig.savefig(f"plots/{exp_id}/sst_nemo.pdf", bbox_inches="tight")
pplt.close(fig)

In [ ]:
nemo_t_grid_cvg = nemo_t_grids[-1]
errors = [
    np.linalg.norm(
        (nemo_t_grid.sosstsst[:, 1, 1] - nemo_t_grid_cvg.sosstsst[:, 1, 1]).data,
        ord=np.inf,
    )
    for nemo_t_grid in nemo_t_grids[:-1]
]
fig, ax = pplt.subplots()
fig.set_size_inches(10, 6)
x_data = np.arange(1, max_iters)
ax.semilogy(x_data, errors, color="k", ls="none", marker=".")
ax.set_xlabel("Schwarz Iteration")
ax.format(
    title="Errors in SST (NEMO)",
    xlabel="Schwarz Iteration",
    ylabel=r"$||e||_\infty$",
    xlocator=x_data[::2],
    xminorlocator=1,
)
fig.savefig(
    f"plots/{exp_id}/errors_linf_sst_nemo.pdf",
    bbox_inches="tight",
)
pplt.close(fig)

# Surface SH Flux

In [ ]:
fig, ax = pplt.subplots()
fig.set_size_inches(10, 6)
for oifs_diagvar in oifs_diagvars[1:]:
    ax.plot(oifs_diagvar.sfc_sen_flx, ls="-", alpha=alpha, color="k")
ax.plot(oifs_diagvars[0].sfc_sen_flx, ls="-", alpha=alpha)
ax.format(
    title="Surface Sensible Heat Flux (OIFS)",
    xlabel="Time",
    ylabel="Heat Flux $[W m^{-2}]$",
)
fig.savefig(f"plots/{exp_id}/ssh_oifs.pdf", bbox_inches="tight")
pplt.close(fig)